# 4 - Data Prep

### 4.1 - Tratamento de missings

&emsp; O problema de missings é prevalente na maioria das áreas de pesquisa. Missing data poduiz inúmeros problemas:

* 1 - reduz o poder dos modelos estatísticos;
* 2 - pode causar viés no modelo;
* 3 - muitos pacotes de modelos de machine learning em python não aceitam missing data. É necessário que os dados sejam tratados primeiramente.

#### 4.1.1 - Missing data mechanisms

* **Missing Completely Random (MCAR)**: se os dados são completamente não relacionados para tanto observados quanto para missing instances.

* **Missing at Random (MAR)**: é quando o missing data é relacionada ao dado observado, mas não a missing data.

* **Missing Not At Random (MNAR)**: são os dados que não são MCAR e MAR. Isso implica que o missing data é relacionado tanto para os observados quanto para missing instances.

#### 4.1.2 - Handling Missing Data

* 1 - Droping Variables;
* 2 - Partial Deletion;
* 3 - Data Imputation;

### 4.2 - Droping Variables

* Exclui a coluna se possui mais de **70% de missing values** caso contrário, Data Imputation é o método mais aplicável do que simplesmente deletar. Quanto maior a informação para o modelo, maior é a confiabilidade dos resultados do modelo. 


In [ ]:
def find_missing_percent(data):
    """
    Returns dataframe containing the total missing values and percentage of total
    missing values of a column.
    """
    miss_df = pd.DataFrame({'ColumnName':[],'TotalMissingVals':[],'PercentMissing':[]})
    for col in data.columns:
        sum_miss_val = data[col].isnull().sum()
        percent_miss_val = round((sum_miss_val/data.shape[0])*100,2)
        miss_df = miss_df.append(dict(zip(miss_df.columns,[col,sum_miss_val,percent_miss_val])),ignore_index=True)
    return miss_df

drop_cols = df[df['PercentMissing'] >70.0].ColumnName.tolist()


### 4.3 - Exclusão Parcial

#### 4.3.1 - Listwise Deletion:
* É uma técnica na qual as linhas que contém missing data são deletadas.

* Desvantagens: diminui a intensidade dos testes estatísticos realizados. Mais amostras de dados são o fator-chave na análise. Como a listwise deletion elimina os dados com valores ausentes, diminui o tamanho dos dados. A listwise deletion pode causar distorções nos dados.

In [3]:
def listwise_deletion(train):
    for col in train.columns:
        miss_ind = train[col][train[col].isnull()].index
        train = train.drop(miss_ind, axis = 0)
    return train

### 4.4 - Data Imputation

#### 4.4.1 - Single Imputation

* A Single Imputation tenta imputar os dados ausentes por um único valor, em oposição à Multiple Imputation, que substitui os dados ausentes por vários valores.

* **Mean Imputation**: é o processo de imputar o missing data pela média da variável e pode ser feito apenas para colunas numéricas.

* **Desvantagens**: mean imputation é mais provável introduzir viés no modelo.

In [4]:
def mean_imputation(train_numeric):
    """
    Mean Imputation
    """
    for col in train_numeric.columns:
        mean = train_numeric[col].mean()
        train_numeric[col] = train_numeric[col].fillna(mean)
    return train_numeric

#### 4.4.2 - Regression Imputation
* Um modelo de regressão é ajustado onde os previsores são os recursos sem missing values e os destinos são os recursos com missing values.

* Os missing values são então substituídos com as previsões. Regression Imputation tem uma chance menor que introduzir viés ao modelo.

In [10]:
def find_missing_index(train_numeric_regr, target_cols):
    """
    Returns the index of the missing values in the columns.
    """
    miss_index_dict = {}
    for tcol in target_cols:
        index = train_numeric_regr[tcol][train_numeric_regr[tcol].isnull()].index
        miss_index_dict[tcol] = index
    return miss_index_dict

def regression_imputation(train_numeric_regr, target_cols, miss_index_dict):
    """
    Fits XGBoost Regressor and replaces the missing values with
    the prediction.
    """
    for tcol in target_cols:
        y = train_numeric_regr[tcol]
        '''Initially impute the column with mean'''
        y = y.fillna(y.mean())
        xgb = xgboost.XGBRegressor(objective="reg:squarederror", random_state=42)
        '''Fit the model where y is the target column which is to be imputed'''
        xgb.fit(predictors, y)
        predictions = pd.Series(xgb.predict(predictors),index= y.index)    
        index = miss_index_dict[tcol]
        '''Replace the missing values with the predictions'''
        train_numeric_regr[tcol].loc[index] = predictions.loc[index]
    return train_numeric_regr

#### 4.4.3 - Mode Imputation
* É o processo de imputar missing data pela moda da variável e pode ser feito somente colunas categoricas.

* **Desvantagem**: chance maior de introduzir viés ao modelo

In [11]:
def mode_imputation(train_categoric):
    """
    Mode Imputation
    """
    for col in train_categoric.columns:
        mode = train_categoric[col].mode().iloc[0]
        train_categoric[col] = train_categoric[col].fillna(mode)
    return train_categoric

#### 4.4.4 - Multiple Imputation
* A missing data pe imputada com multiploes valores inúmeras vezes em uma multitude de conjunto de dados imputados.


* **MICE (Multiple Imputation by Chained Equation)**:
    * Passo 01: Inicialmente, o conjunto de dados é imputado com a média agindo como um 'placeholder';
    * Passo 02: Uma missing value é escolhida aleatóriamente e entitulada como o target. Um modelo de regressão é treinado nessas features e no target;
    * Passo 03: As missings values do target são então subtituídas com previsões (imputações) do modelo de regressão;
    * Passo 04: Os passos 02 e 03 são então repetidos para cada feature que tenha dados não observados. Ao fim de um ciclo, todos os missing values terão sido imputados com as previsões;
    * Passo 05: Os passos 02 e 03 são então repetidos por um número específico de ciclos;
    
    
* **Algoritmo MICE para dados categóricos**: antes de usarmos os passos acima precisamos seguir os passos abaixo em ordem de imputar dados categóricos:

    * Passo 01: Encode Ordinal de valores non-null;
    * Passo 02: Usar MICE com Gradient Boosting Classifier para imputar dados do Encode Ordinal;
    * Passo 03: Converter os valores de ordinais de volta para valores categoricos;
    * Passo 04: Seguir os passos 01 ao 05 do MICE. Ao invés de usar Mean Imputation como estratégia inicial usaremos Mode Imputation;

In [ ]:
def mice_imputation_numeric(train_numeric):
    """
    Impute numeric data using MICE imputation with Gradient Boosting Regressor.
    (we can use any other regressors to impute the data)
    """
    iter_imp_numeric = IterativeImputer(GradientBoostingRegressor())
    imputed_train = iter_imp_numeric.fit_transform(train_numeric)
    train_numeric_imp = pd.DataFrame(imputed_train, columns = train_numeric.columns, index= train_numeric.index)
    return train_numeric_imp

def mice_imputation_categoric(train_categoric):
    """
    Impute categoric data using MICE imputation with Gradient Boosting Classifier.
    Steps:
    1. Ordinal Encode the non-null values
    2. Use MICE imputation with Gradient Boosting Classifier to impute the ordinal encoded data
    (we can use any other classifier to impute the data)
    3. Inverse transform the ordinal encoded data.
    """
    ordinal_dict={}
    for col in train_categoric:
        '''Ordinal encode train data'''
        ordinal_dict[col] = OrdinalEncoder()
        nn_vals = np.array(train_categoric[col][train_categoric[col].notnull()]).reshape(-1,1)
        nn_vals_arr = np.array(ordinal_dict[col].fit_transform(nn_vals)).reshape(-1,)
        train_categoric[col].loc[train_categoric[col].notnull()] = nn_vals_arr

    '''Impute the data using MICE with Gradient Boosting Classifier'''
    iter_imp_categoric = IterativeImputer(GradientBoostingClassifier(), max_iter =5, initial_strategy='most_frequent')
    imputed_train = iter_imp_categoric.fit_transform(train_categoric)
    train_categoric_imp = pd.DataFrame(imputed_train, columns =train_categoric.columns,index = train_categoric.index).astype(int)
    
    '''Inverse Transform'''
    for col in train_categoric_imp.columns:
        oe = ordinal_dict[col]
        train_arr= np.array(train_categoric_imp[col]).reshape(-1,1)
        train_categoric_imp[col] = oe.inverse_transform(train_arr)
        
    return train_categoric_imp

## 4.2 - Tratamento de Outliers

Os outliers se referem ao pontos que estão muito distantes de onde os demais estão contidos. Ou seja, outliers são pontos raros ou distintos. Podemos usar 3 maneiras de trabalhar com eles, essas sendo:

### 4.2.1 - Univariated Method
* Um dos métodos mais simples para detecção de outliers, usando box plots. Um box plot pode ser descrito como uma representação gráfica da distribuição dos dados. Usando mediana e os interquartis. A distância máxima do centro dos dados que vai ser permitido é chamado de **cleaning parameter**. Se o **cleaning parameter** é extenso, o teste se torna menos sensitivo a outliers. Caso seja muito pequeno, muitos valores são detectados com outliers.

### 4.2.2 - Multivariated Method
* Ouliers não precisam ser valores extremos. Na verdade, as vezes o método anterior não funciona muito bem. O Multivariated Method tenta resolver criando um modelo preditivo usando todas os dados deisponíveis e limpando as instâncias com erros acima do valor dado. 

### 4.2.3 - Minkowski Error
* Diferente dos métodos anteriores o Minkowski error não detecta ou limpa os outliers, ao invés ele reduz o impacto que eles terão no modelo.
* O Minkowski Error é um Loss Index que é mais sensível aos outliers que para o MSE, que levanta cada erro da instância ao quadrado ocasionando os outliers a terem uma grande contribuição, No método Minkwski Error temos a solução onde ao invés de elevarmos ao quadrado elevamos a um número menor que 2. Esse número é chamado de parâmetro Minkowski, e reduz a contribuição dos outliers para o erro total, dando a seguinte equação: $\text{minkowski_error} = \frac{\sum(\text{outputs - targets})^{minkowski_parameter}}{\text{instances_number}}$